# Import packages

In [ ]:
## IMPORTANT - NOTEBOOK

import pandas as pd
from teradataml import *

import ast
import json


from utils import clean_text
from constants import (

SEUODI_DATASET_MAPPED, BRAND_DB ,JIO_MART_DATASET, JIO_MART_DATASET_MAPPED, JIO_MART_DATASET_MAPPING, BIG_BASKET_DATASET, FLIP_KART_DATASET, MWPD_FULL_DATASET
)


In [16]:
mwpd = pd.read_csv(MWPD_FULL_DATASET)
mwpd.head()

,Name,Description,SegmentTitle,FamilyTitle,ClassTitle
0,Sterling Silver Angel Charm,This little angel charm is just heavenly,64000000_Personal Accessories,64010000_Personal Accessories,64010100_Jewellery
1,HP Pavilion 23xi 58.40 cm (23�) IPS Monitor,"Share photos, videos, and games with everyone ...",65000000_Computing,65010000_Computers/Video Games,65010700_Computer/Video Game Peripherals
2,East Carolina Pirates Ladies Personalized Bask...,Feel like a bona fide member of East Carolina ...,67000000_Clothing,67010000_Clothing,67010800_Upper Body Wear/Tops
3,"Tekonsha 90195 P3 Electric Brake Control, 1-4 ...",Receive free shipping on this item. Enter coup...,77000000_Automotive,77010000_Automotive Accessories and Maintenance,77011200_Automotive Maintenance/Repair
4,RN-XV WiFly Module - Wire Antenna,Description: The RN-XV module by Roving Networ...,78000000_Electrical Supplies,78050000_Electronic Communication Components,78050100_Electronic Communication Components


In [18]:
mwpd_classes = mwpd["ClassTitle"].unique()
mwpd_classes

array(['64010100_Jewellery', '65010700_Computer/Video Game Peripherals',
       '67010800_Upper Body Wear/Tops',
       '77011200_Automotive Maintenance/Repair',
       '78050100_Electronic Communication Components',
       '51150100_Medical Devices', '75030100_Ornaments',
       '71010300_Racquet Sports Equipment',
       '75030200_Pictures/Mirrors/Frames',
       '71010800_Cycle Sports Equipment',
       '81010100_Lawn/Garden Pools/Ponds/Water Features and Ornaments',
       '75010200_Household/Office Seating',
       '71010200_Sports Balls/Pucks/Shuttlecocks/Frisbees/Boomerangs',
       '51102100_Sensory Organs Care/Treatments',
       '62060700_Stationery Adhesives/Binders/Fasteners',
       '63010400_Indoor Footwear', '67010900_Protective Wear',
       '67010200_Full Body Wear',
       '71011800_Watercraft Sports Equipment (Non Powered)',
       '67010100_Clothing Accessories', '68020100_Photography',
       '68040100_Pre-Recorded or Digital Content Media',
       '80011600_Hammer

## Jio Mart

In [4]:
jio_mart_df = pd.read_csv(JIO_MART_DATASET)
jio_mart_df

,category,sub_category,href,items,price
0,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Fresh Dates (Pack) (Approx 450 g - 500 g),109.0
1,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,49.0
2,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Mosambi 1 kg,69.0
3,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Orange Imported 1 kg,125.0
4,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,44.0
...,...,...,...,...,...
162308,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 5.6 gm Ganesha Silver ...,1669.0
162309,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 4.58 gm Ganesha Silver...,1391.0
162310,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 2.31 gm Ganesha Silver...,834.0
162311,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 7.76 gm Ganesha Silver...,2276.0


In [5]:
jio_mart_df = pd.read_csv(JIO_MART_DATASET)
jio_mart_df = jio_mart_df.loc[~jio_mart_df['items'].isna(), :]
jio_mart_df = jio_mart_df.drop(columns=['href', 'price'])
jio_mart_df.rename(columns={'items': 'product name'}, inplace=True)
jio_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162280 entries, 0 to 162312
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      162280 non-null  object
 1   sub_category  162280 non-null  object
 2   product name  162280 non-null  object
dtypes: object(3)
memory usage: 5.0+ MB


In [6]:
jio_mart_df.head()

,category,sub_category,product name
0,Groceries,Fruits & Vegetables,Fresh Dates (Pack) (Approx 450 g - 500 g)
1,Groceries,Fruits & Vegetables,Tender Coconut Cling Wrapped (1 pc) (Approx 90...
2,Groceries,Fruits & Vegetables,Mosambi 1 kg
3,Groceries,Fruits & Vegetables,Orange Imported 1 kg
4,Groceries,Fruits & Vegetables,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...


In [7]:
jio_mart_df_cateogires = jio_mart_df["sub_category"].unique()
jio_mart_df_cateogires

array(['Fruits & Vegetables', 'Premium Fruits', 'Dairy & Bakery',
       'Staples', 'Snacks & Branded Foods', 'Beverages', 'Personal Care',
       'Home Care', 'Apparel', 'Mom & Baby Care', 'Books', 'Pets',
       'Kitchenware', 'Dining', 'Furnishing', 'Home Decor', 'Furniture',
       'Home Appliances', 'Toys, Games & Fitness', 'Electrical',
       'Bathroom & Laundry Accessories', 'Disposables', 'Stationery',
       'Bags & Travel Luggage', 'Mops, Brushes & Scrubs', 'Auto Care',
       'Garden & Outdoor', 'Carpentry & work accessories', 'Pooja Needs',
       'Bathroom & Laundry', 'Industrial & Scientific Supplies',
       'Building Supplies & Measuring Tools', 'Hardware & Plumbing',
       'Home Safety & Automation', 'Kitchen & Bath Fixtures',
       'Paint, Wall Treatments & Supplies', 'Power & Hand Tools',
       'Handloom & Handicraft', 'Personal Wear', 'Men', 'Women', 'Boys',
       'Girls', 'Junior Boys', 'Infants', 'Mobiles & Tablets',
       'TV & Speaker', 'Computers', 'Camer

In [8]:
with open(JIO_MART_DATASET_MAPPING, "r", encoding="utf-8") as f:
    mapping = json.load(f)

In [9]:
def map_to_gpc(category, subcategory):
    try:
        return mapping[category][subcategory]
    except KeyError:
        return [None, None, None, None]

In [10]:
jio_mart_df[["Segment", "Family", "Class", "Brick"]] = jio_mart_df.apply(
    lambda row: pd.Series(map_to_gpc(row["category"], row["sub_category"])),
    axis=1
)

In [11]:
jio_mart_df.to_csv(JIO_MART_DATASET_MAPPED, index=False)

In [12]:
jio_mart_df.head()

,category,sub_category,product name,Segment,Family,Class,Brick
0,Groceries,Fruits & Vegetables,Fresh Dates (Pack) (Approx 450 g - 500 g),Food/Beverage,Fruits/Vegetables Fresh Cut,NaN,NaN
1,Groceries,Fruits & Vegetables,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,Food/Beverage,Fruits/Vegetables Fresh Cut,NaN,NaN
2,Groceries,Fruits & Vegetables,Mosambi 1 kg,Food/Beverage,Fruits/Vegetables Fresh Cut,NaN,NaN
3,Groceries,Fruits & Vegetables,Orange Imported 1 kg,Food/Beverage,Fruits/Vegetables Fresh Cut,NaN,NaN
4,Groceries,Fruits & Vegetables,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,Food/Beverage,Fruits/Vegetables Fresh Cut,NaN,NaN


## Big Market

In [11]:
big_mart_df = pd.read_csv(BIG_BASKET_DATASET)
big_mart_df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
big_mart_df = big_mart_df.loc[~jio_mart_df['product'].isna(), :]
big_mart_df = big_mart_df.drop(columns=['brand', 'sale_price', 'market_price', 'rating','index'])
big_mart_df.rename(columns={'product': 'product name', 'category' : 'level_1', 'sub_category' : 'level_2', 'type' : 'level_3'}, inplace=True)
big_mart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27555 entries, 0 to 27554
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product name  27554 non-null  object
 1   level_1       27555 non-null  object
 2   level_2       27555 non-null  object
 3   level_3       27555 non-null  object
 4   description   27440 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [13]:
big_mart_df.head()

,product name,level_1,level_2,level_3,description
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Hair Oil & Serum,This Product contains Garlic Oil that is known...
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Water & Fridge Bottles,"Each product is microwave safe (without lid), ..."
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Lamp & Lamp Oil,"A perfect gift for all occasions, be it your m..."
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,"Laundry, Storage Baskets",Multipurpose container with an attractive desi...
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Bathing Bars & Soaps,Nivea Creme Soft Soap gives your skin the best...


In [21]:
big_mart_df_level_1 = big_mart_df["level_1"].unique()
big_mart_df_level_1

array(['Beauty & Hygiene', 'Kitchen, Garden & Pets',
       'Cleaning & Household', 'Gourmet & World Food',
       'Foodgrains, Oil & Masala', 'Snacks & Branded Foods', 'Beverages',
       'Bakery, Cakes & Dairy', 'Baby Care', 'Fruits & Vegetables',
       'Eggs, Meat & Fish'], dtype=object)

In [19]:
big_mart_df_level_2 = big_mart_df["level_2"].nunique()
big_mart_df_level_2

90

In [18]:
big_mart_df_level_3 = big_mart_df["level_3"].nunique()
big_mart_df_level_3

426

## Flip Kart

In [2]:
flip_kart_df = pd.read_csv(FLIP_KART_DATASET)
flip_kart_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
flip_kart_df = flip_kart_df.loc[~flip_kart_df['product_name'].isna(), :]
flip_kart_df = flip_kart_df.drop(columns=['uniq_id', 'product_url', 'crawl_timestamp', 'pid','retail_price', 'discounted_price', 'image', 'is_FK_Advantage_product', 'product_rating', 'overall_rating', 'brand', 'product_specifications'])
flip_kart_df.head()

,product_name,product_category_tree,description
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...


In [17]:
flip_kart_df_tree = flip_kart_df["product_category_tree"].iloc[7867]
flip_kart_df_tree

'["Home Decor & Festive Needs >> Wall Decor & Clocks >> Clocks >> Wall Clocks >> Blacksmith Wall Clocks"]'

In [19]:
def extract_levels(category_tree_str):
   if pd.isna(category_tree_str):
       return [None, None, None, None]
   
   try:
       clean_str = category_tree_str.strip('[""]')
       levels = clean_str.split(' >> ')
       result = (levels + [None, None, None, None])[:4]
       return result
   except:
       return [None, None, None, None]

In [20]:
flip_kart_df[['level_1', 'level_2', 'level_3', 'level_4']] = pd.DataFrame(
   flip_kart_df['product_category_tree'].apply(extract_levels).tolist(), 
   index=flip_kart_df.index
)
flip_kart_df.head()

,product_name,product_category_tree,description,level_1,level_2,level_3,level_4
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,Furniture,Living Room Furniture,Sofa Beds & Futons,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,Footwear,Women's Footwear,Ballerinas,AW Bellies
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Pet Supplies,Grooming,Skin & Coat Care,Shampoo


## Seoudi Dataset

In [10]:
seoudi_df = pd.read_csv(SEUODI_DATASET_MAPPED)
seoudi_df.head()

,id,name,brand,url,level_1,level_2,level_3
0,8859258,"Flamer Stainless Steel Fork, Small - A-360",NaN,flamer-stainless-steel-fork-small-a-360,"Home, Kitchen & Garden",Kitchen & Dining,Cutlery
1,7076761,"Litchi Stainless Steel Thermal Mug F-195 A, Bl...",Litchi,litchi-stainless-steel-thermal-mug-f195-a-blac...,Top Offers,Thermal mugs & Bottles,Thermal Mugs
2,125883,Best Female Plug - 16 A,Best,best-female-plug-16-a,Electronics & Appliances,Batteries & Power,Sockets & Plugs
3,59784,Class A Caramel Biscuits - 37 g,NaN,class-a-caramel-biscuits-37-g,Top Offers,Biscuits & Cookies,Biscuits
4,56844,Class A Medium Couscous - 400 g,Class A,class-a-medium-couscous-400-g,Food Cupboard,Sugar & Home Baking,Dessert Mix


In [11]:
seoudi_df = seoudi_df.loc[~seoudi_df['name'].isna(), :]
seoudi_df = seoudi_df.drop(columns=['brand', 'id', 'url'])
seoudi_df.rename(columns={'name': 'product_name'}, inplace=True)
seoudi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19966 entries, 0 to 19965
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  19966 non-null  object
 1   level_1       19961 non-null  object
 2   level_2       19838 non-null  object
 3   level_3       18202 non-null  object
dtypes: object(4)
memory usage: 779.9+ KB


In [19]:
seoudi_df_l_1 = seoudi_df["level_1"].nunique()
seoudi_df_l_1

24

In [20]:
seoudi_df_l_2 = seoudi_df["level_2"].nunique()
seoudi_df_l_2

231

In [21]:
seoudi_df_l_3 = seoudi_df["level_3"].nunique()
seoudi_df_l_3

526

In [13]:
import chardet

with open(BRAND_DB, 'rb') as f:
    raw = f.read(100000)  
    result = chardet.detect(raw)

enc = result['encoding']

df = pd.read_csv(BRAND_DB, encoding=enc)
df.head()

,Sector,Brand,Product,Segment,Family,Class
0,Food and Beverages,Coca-Cola,Coca-Cola (Classic),Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
1,Food and Beverages,Coca-Cola,Diet Coke,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
2,Food and Beverages,Coca-Cola,Coca-Cola Zero Sugar,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
3,Food and Beverages,Coca-Cola,Coca-Cola Cherry,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
4,Food and Beverages,Coca-Cola,Coca-Cola Vanilla,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink


In [14]:
mask_empty_or_na = df["Segment"].isnull() | df["Segment"].astype(str).str.strip().eq("") | df["Segment"].isin(["None", "NULL", "NA"])
null_segment_df2 = df[mask_empty_or_na]
null_segment_df2.isnull().sum()

Sector        0
Brand         0
Product       0
Segment    1640
Family     1640
Class      1640
dtype: int64

In [21]:
non_null_segment_df = df[~mask_empty_or_na]
non_null_segment_df.head()

,Sector,Brand,Product,Segment,Family,Class
0,Food and Beverages,Coca-Cola,Coca-Cola (Classic),Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
1,Food and Beverages,Coca-Cola,Diet Coke,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
2,Food and Beverages,Coca-Cola,Coca-Cola Zero Sugar,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
3,Food and Beverages,Coca-Cola,Coca-Cola Cherry,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink
4,Food and Beverages,Coca-Cola,Coca-Cola Vanilla,Food/Beverage,Beverages,Non Alcoholic Beverages - Ready to Drink


In [23]:
non_null_segment_df.to_csv("already_labblled_brands.csv")

In [15]:
# brand_df = brand_df[brand_df["Segment"].isna()]
null_segment_df2.to_csv("part1.csv")

In [ ]:
brand_df_brands = brand_df["Brand"].unique()
brand_df_brands

In [ ]:
brand_products = brand_df.groupby("Brand")["Product"].apply(list).reset_index()
brand_products.to_csv("brands.csv")

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("already_labblled_brands.csv")
import chardet

with open("part1_labbelled.csv", 'rb') as f:
    raw = f.read(100000)  
    result = chardet.detect(raw)

enc = result['encoding']

df2 = pd.read_csv("part1_labbelled.csv", encoding=enc)
df_output = pd.concat([df, df2], ignore_index=True)

In [6]:
df_output.to_csv("brand_dataset_half_mapped_to_gpc.csv", index=False)

In [10]:
df_analysis = pd.read_csv("class_only_predictions.csv")
df_wrong = df_analysis[df_analysis["segment"] != df_analysis["pred_segment"]]
df_wrong2 = df_wrong.tail(100)
df_wrong2.head(30)

,Unnamed: 0,product_name,description,segment,family,class,pred_segment,pred_family,pred_class
12494,20243,Klassiker Pasta Sauce Tomate,NaN,food beverage,seasonings preservatives extracts,sauces spreads dips condiments,food beverage tobacco,prepared preserved foods,ready made combination meals
12499,13677,Mens Crazyquick,ADIDAS Crazyquick 2.0 Men's Football Cleat The...,footwear,footwear,athletic footwear,food beverage,cereal grain pulse products,breakfast cereals
12505,2639,Original Menthol Flavor E Liquid EJuice Manufa...,Basic Info Product Description Model NO.: TE-1...,food beverage tobacco,tobacco smoking accessories,tobacco products smoking accessories,safety protection diy,safety protection diy,protective personal aids diy
12601,10438,Principles for a Successful Life From Our Fami...,Is life really so complicated? Ask happy and s...,audio visual photography,audio visual media,pre recorded or digital content media,textual printed reference materials,textual printed reference materials,books
12611,53222,Early Foods Gluten Free Biscuits Combo Pack O...,NaN,beauty personal care hygiene,personal hygiene products,baby diapers accessories,food beverage,prepared preserved foods,snacks
12623,13746,Glo Head Ape Man Model Kit,Glo Head Ape Man Model Kit Glo Head Ape Man Mo...,arts crafts needlework,arts crafts needlework supplies,needlework toy making craft supplies,toys games,toys games,dolls puppets action figures soft toys
12638,29895,Jiwa Rolled Oats g,NaN,food beverage,prepared preserved foods,snacks,food beverage tobacco,bread bakery products,sweet bakery products
12754,9167,Puptron Silver Tequila Squeaky Dog Toy,Original Puptron Silver Tequila Plush Squeaky ...,sports equipment,sports equipment,sports equipment accessories,pet care food,pet care,pet accessories
12774,59213,Brainy Bear Early Skills Subscription Box Months,NaN,beauty personal care hygiene,personal hygiene products,baby diapers accessories,computing,computers video games,computer video game software
12802,173236,Organic Harvest Cold Pressed Pure Argan Oil ml,NaN,beauty personal care hygiene,hair products,hair care products,food beverage,vegetables non leaf unprepared unprocessed fresh,root tuber vegetables


In [2]:
df = pd.read_csv("../all_data_0.85.csv")
df.head()

,Unnamed: 0,product_name,description,segment,family,class,brick,source,text,dedup_key
0,0,Sterling Silver Angel Charm,This little angel charm is just heavenly,Personal Accessories,Personal Accessories,Jewellery,NaN,MWPD_FULL,sterling silver angel charm,sterling silver angel charm
1,1,HP Pavilion xi . cm (�) IPS Monitor,"Share photos, videos, and games with everyone ...",Computing,Computers/Video Games,Computer/Video Game Peripherals,NaN,MWPD_FULL,hp pavilion xi . cm ips monitor,hp pavilion xi cm ips monitor
2,2,East Carolina Pirates Ladies Personalized Bask...,Feel like a bona fide member of East Carolina ...,Clothing,Clothing,Upper Body Wear/Tops,NaN,MWPD_FULL,east carolina pirates ladies personalized bask...,east carolina pirates ladies personalized bask...
3,3,"Tekonsha P Electric Brake Control, - Trailer ...",Receive free shipping on this item. Enter coup...,Automotive,Automotive Accessories and Maintenance,Automotive Maintenance/Repair,NaN,MWPD_FULL,tekonsha p electric brake control - trailer axles,tekonsha p electric brake control trailer axles
4,4,RN-XV WiFly Module - Wire Antenna,Description: The RN-XV module by Roving Networ...,Electrical Supplies,Electronic Communication Components,Electronic Communication Components,NaN,MWPD_FULL,rn-xv wifly module - wire antenna,rn xv wifly module wire antenna


In [3]:
from teradataml.dataframe.copy_to import copy_to_sql

In [4]:
from modules.db import TeradataDatabase

In [5]:
td_db = TeradataDatabase() 
td_db.connect()

In [6]:
copy_to_sql(df, "full_dataset_un_cleaned", "demo_user", if_exists="replace")

TeradataMlException: [Teradata][teradataml](TDML_2020) Failed to copy dataframe to Teradata Vantage.(teradatasql.OperationalError) [Version 20.0.0.34] [Session 3360] [Teradata SQL Driver] [Error 528] A failure occurred while executing rows 15537 through 16383 of a batch request. Details of the failure can be found in the error string that is accessible using the escape functions {fn teradata_nativesql}{fn teradata_get_errors}.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:83
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverError ErrorUtil.go:150
 at gosqldriver/teradatasql.(*TeradataRows).generateBatchErrors TeradataRows.go:1128
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:730
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:823
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1786
 at databa